In [2]:
!pip install mysql-connector-python
!pip install pandas


In [2]:
import mysql.connector##Importing required libraries
from mysql.connector import Error
import pandas as pd


In [3]:
def create_server_connection(host_name, user_name, user_password):#Creating a method to to connect to the server
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection
pw = "mysqlpass"

In [5]:
connection = create_server_connection("localhost", "root", pw)#Function to use later to direct queries to database

MySQL Database connection successful


In [6]:
def create_database(connection, query):##Creating a method to thats handles the database creation query
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")

In [7]:
create_database_query = "CREATE DATABASE mall"
create_database(connection, create_database_query)## defining and creating mall database

Error: '1007 (HY000): Can't create database 'mall'; database exists'


In [7]:
def create_db_connection(host_name, user_name, user_password, db_name):## Creating a method to connect to the database
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [8]:
def execute_query(connection, query):## This will handle and send queries to the cursor for execution in mysql
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [54]:
## Defining our table values and data types
##In customer table customer_id is primary key
create_customer_table = """
CREATE TABLE customer (
  customer_id INT PRIMARY KEY, 
  first_name VARCHAR(40) NOT NULL,
  last_name VARCHAR(40) NOT NULL,
  registration_date DATE,
  score INT
  );
 """ 
##In inventory table item_id is primary key
create_inventory_table = """
CREATE TABLE inventory (
  item_id INT PRIMARY KEY,
  item_name VARCHAR(40) NOT NULL,
  cost INT
  );
 """
##In purchase table customer_id and item_id are primary keys because a purchase cannot be made without knowing the customer who is buying and the product being bought
create_purchase_table = """
CREATE TABLE purchase(
customer_id INT,
item_id INT,
FOREIGN KEY(customer_id) REFERENCES customer(customer_id),
FOREIGN KEY(item_id) REFERENCES inventory(item_id)
);
"""
connection = create_db_connection("localhost", "root", pw, "mall")## database to select
execute_query(connection, create_customer_table)
execute_query(connection, create_inventory_table)## execute query in said database
execute_query(connection, create_purchase_table)

MySQL Database connection successful
Query successful


In [55]:
#Populating tables
pop_customer = """
INSERT INTO customer VALUES
(1, "James", "Smith", "2020-04-20", 15),
(2, "Stefanie", "Martin", "2020-02-17", 50), 
(3, "Steve", "Wang", "2020-11-12", 65),
(4, "Friederike",  "Müller-Rossi", "2020-07-07", 80),
(5, "Isobel", "Ivanova", "2020-05-30", 70);
"""
pop_inventory = """
INSERT INTO inventory VALUES
(101, 'lollipop' , 50),
(102, 'shirt',70),
(103, 'laptop',100),
(104, 'tomatoe',60),
(105, 'apple',250);
"""

pop_purchase = """
INSERT INTO purchase VALUES
(1, 101),
(2, 102),
(3, 103),
(4, 104),
(5, 105);
"""
connection = create_db_connection("localhost", "root", pw, "mall")
execute_query(connection, pop_customer)
execute_query(connection, pop_inventory)#Sending Quearies to mall database
execute_query(connection, pop_purchase)

MySQL Database connection successful
Query successful


In [9]:
def read_query(connection, query):# This method will execute query and return the results
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

In [10]:
#Solving Question 4a
#SELECT<WHERE score < 20>(customer)
q1 = """
SELECT *
FROM customer
WHERE score < 20;
"""



connection = create_db_connection("localhost", "root", pw, "mall")
results = read_query(connection, q1)

#Thsi will return a list of lists and then create a pandas DataFrame
from_db = []

for result in results:
  result = list(result)
  from_db.append(result)


columns = ["customer_id", "first_name", "last_name", "registration_date", "score"]
df = pd.DataFrame(from_db, columns=columns)

display(df)

MySQL Database connection successful


,customer_id,first_name,last_name,registration_date,score
0,1,James,Smith,2020-04-20,15


In [11]:
# Question 4b
#SELECT<WHERE score >= 50>(customer)
q2 = """
SELECT *
FROM customer
WHERE score >= 50;
"""
connection = create_db_connection("localhost", "root", pw, "mall")
results = read_query(connection, q2)

from_db = []

for result in results:
  result = list(result)
  from_db.append(result)


columns = ["customer_id", "first_name", "last_name", "registration_date", "score"]
df = pd.DataFrame(from_db, columns=columns)

display(df)

MySQL Database connection successful


,customer_id,first_name,last_name,registration_date,score
0,2,Stefanie,Martin,2020-02-17,50
1,3,Steve,Wang,2020-11-12,65
2,4,Friederike,Müller-Rossi,2020-07-07,80
3,5,Isobel,Ivanova,2020-05-30,70


In [12]:
##Question 4c
#One <- SELECT<WHERE score >=50>(customer)
#Two <- PROJECT<first_name, last_name>(One)
q3 = """
SELECT first_name, last_name
FROM customer
WHERE score >= 50;
"""
connection = create_db_connection("localhost", "root", pw, "mall")
results = read_query(connection, q3)

from_db = []

for result in results:
  result = list(result)
  from_db.append(result)


columns = [ "first_name", "last_name"]
df = pd.DataFrame(from_db, columns=columns)

display(df)

MySQL Database connection successful


,first_name,last_name
0,Stefanie,Martin
1,Steve,Wang
2,Friederike,Müller-Rossi
3,Isobel,Ivanova


In [24]:
#Question 4d
#RESULT1 <- SELECT<WHERE score < 20>(customer)
#RESULT2 <- SELECT<WHERE score >= 50>(customer)
#RESULT <- RESULT1 UNION RESULT2
q4 = """
SELECT first_name, last_name
FROM customer
WHERE score < 20
UNION
SELECT first_name, last_name
FROM customer
WHERE score >= 50;

"""
operation = q4
cursor = connection.cursor()

        
from_db = []

for result in cursor.execute(operation, multi=True):## We need to execute with multi=True because of UNION function
    if result.with_rows:
        result = list(result)
        from_db.append(result)
        columns = ["first_entity", "second_entity", "third_entity", "fourth_entity", "fifth_entity"]
        df = pd.DataFrame(from_db, columns=columns)
        display(df)
        
    else:
        print("Number of Rows affected '{}': {}".format(
        result.statement, result.rowcount))
                        

,first_entity,second_entity,third_entity,fourth_entity,fifth_entity
0,"(James, Smith)","(Stefanie, Martin)","(Steve, Wang)","(Friederike, Müller-Rossi)","(Isobel, Ivanova)"


In [25]:
##Question 4e
#RESULT <- CUSTOMER X PURCHASE
q5 = """
SELECT *
FROM customer
CROSS JOIN
purchase
"""
connection = create_db_connection("localhost", "root", pw, "mall")
results = read_query(connection, q5)

from_db = []

for result in results:
  result = list(result)
  from_db.append(result)


columns = ["customer_id", "first_name", "last_name", "registration_date", "score","Customer_id","Item_id"]
df = pd.DataFrame(from_db, columns=columns)

display(df)

MySQL Database connection successful


,customer_id,first_name,last_name,registration_date,score,Customer_id,Item_id
0,1,James,Smith,2020-04-20,15,1,101
1,2,Stefanie,Martin,2020-02-17,50,1,101
2,3,Steve,Wang,2020-11-12,65,1,101
3,4,Friederike,Müller-Rossi,2020-07-07,80,1,101
4,5,Isobel,Ivanova,2020-05-30,70,1,101
5,1,James,Smith,2020-04-20,15,2,102
6,2,Stefanie,Martin,2020-02-17,50,2,102
7,3,Steve,Wang,2020-11-12,65,2,102
8,4,Friederike,Müller-Rossi,2020-07-07,80,2,102
9,5,Isobel,Ivanova,2020-05-30,70,2,102


In [27]:
##Question 4f
##RESULT1 <- SELECT<WHERE score >= 50>(customer)
#NAME <- PROJECT<customer_id>(RESULT1)
##RESULT2 <- SELECT<customer_id>(purchase)
##RESULT <- NAME INSTERSECT RESULT2
q6 = """
SELECT customer_id
FROM customer
WHERE score >= 50
INTERSECT
SELECT customer_id
FROM purchase;

"""
operation = q6
cursor = connection.cursor()
from_db = []

for result in cursor.execute(operation, multi=True):
    if result.with_rows:
        result = list(result)
        from_db.append(result)
        columns = ["First_Entity", "Second_Entity", "Third_Entity", "Fourth_Entity"]
        df = pd.DataFrame(from_db, columns=columns)
        display(df)
        
    else:
        print("Number of Rows affected '{}': {}".format(
        result.statement, result.rowcount))
            


,First_Entity,Second_Entity,Third_Entity,Fourth_Entity
0,"(2,)","(3,)","(4,)","(5,)"


In [31]:
##Question 5
q7 = """
SELECT first_name, last_name
FROM customer
WHERE score > 30
AND customer_id
IN (SELECT customer_id
FROM purchase
WHERE item_id
IN(SELECT item_id
FROM inventory
WHERE cost > 200));

"""
connection = create_db_connection("localhost", "root", pw, "mall")
results = read_query(connection, q7)

from_db = []

for result in results:
    result = list(result)
    from_db.append(result)
    
    
columns = ["first_name", "Last_name"]
df = pd.DataFrame(from_db, columns=columns)
display(df)
        


MySQL Database connection successful


,first_name,Last_name
0,Isobel,Ivanova


In [48]:
##Creating class to insert customer vaues into
class putin:
    connection = create_db_connection("localhost", "root", pw, "mall")
    def __init__(self, connection, value):
        self.connection=connection;
        self.value=value;
        
    def insert_customer(self):
        cursor = connection.cursor()
       
    
        command = self.value;##Setting query
        cursor.execute(command)
        connection.commit()
        print("Inserted into customer")
   ##Test     
s= """INSERT INTO customer VALUES(6, 'qidas', 'fiqas', '2020-8-21', 78);"""##Query content
r = putin(connection, s)##pushing in to class
r.insert_customer()## Using insert method

MySQL Database connection successful
Inserted into customer


In [49]:
class put_in:##Creating class for putting inventory values into
    connection = create_db_connection("localhost", "root", pw, "mall")
    def __init__(self, connection, values):
        self.connection=connection;
        self.values=values;
        
        
    def insert_inventory(self):
        cursor = connection.cursor()
        command = self.values;##Setting query
    
        cursor.execute(command)
        connection.commit()
        print("Inserted into inventory")
  ##Test      
u = """INSERT INTO inventory VALUES(106, 'Alcohol', 100);"""#Query to be inserted
x = put_in(connection, u)
x.insert_inventory()

MySQL Database connection successful
Inserted into inventory


In [50]:
class pUtin:##Class for insertion into purchase tables
    connection = create_db_connection("localhost", "root", pw, "mall")
    def __init__(self, connection, variable):
        self.connection=connection;
        self.variable=variable;
        
        
    def insert_purchase(self):
        cursor = connection.cursor()
        command = self.variable; ##Setting query
    
        cursor.execute(command)
        connection.commit()
        print("Inserted into purchase")
 ##Test       
a = """INSERT INTO purchase VALUES(6, 106);""" ##Query
e = pUtin(connection, a)
e.insert_purchase()

MySQL Database connection successful
Inserted into purchase


In [51]:
##Showing proof that the table values in the previous cell(s) have been added to database
proof = """
SELECT first_name, last_name
FROM customer
WHERE customer_id = 6;
"""
proof2 = """
SELECT item_id, item_name
FROM inventory
WHERE cost = 100;
"""
proof3 = """
SELECT *
FROM purchase
WHERE item_id = 106;
"""
connection = create_db_connection("localhost", "root", pw, "mall")
results = read_query(connection, proof)
##Printing results
for result in results:
  print(result)

results1 = read_query(connection, proof2)

for result in results1:
  print(result)

results2 = read_query(connection, proof3)

for result in results2:
  print(result)

MySQL Database connection successful
('qidas', 'fiqas')
(103, 'laptop')
(106, 'Alcohol')
(6, 106)
